In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.8 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [5]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [8]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [9]:
pipeline=transformers.pipeline("text-generation",
                               model=model,
                               tokenizer=tokenizer,
                               torch_dtype=torch.bfloat16,
                               device_map="auto",
                               max_new_tokens=512,
                               min_new_tokens=-1,
                               temperature=0.75,
                               do_sample=True,
                               top_k=30,
                               num_return_sequences=1,
                               eos_token_id=tokenizer.eos_token_id)

In [10]:
!nvidia-smi

Sat Oct 14 07:27:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    25W /  70W |  13589MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [12]:
B_INST, E_INST= "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [13]:
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""

In [14]:
instruction = "Who won the T20 Cricket World Cup 2022"

In [15]:
SYSTEM_PROMPT=B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [17]:
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [18]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [19]:
template

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\nWho won the T20 Cricket World Cup 2022[/INST]"

In [20]:
instruction = "Convert the following text from English to Tamil: \n\n{text}"

In [21]:
SYSTEM_PROMPT=B_SYS+DEFAULT_SYSTEM_PROMPT+E_SYS

In [22]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [23]:
print(template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Convert the following text from English to Tamil: 

{text}[/INST]


In [24]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [25]:
text ="How are you"

In [26]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [29]:
LLM_Chain.run(text)

'  I\'m just an AI and I don\'t have personal experiences or emotions, so I don\'t have a "how are you" in the classical sense. However, I\'m here to help you with any questions or tasks you may have, so please feel free to ask me anything! இருக்கிறோ?'

In [30]:
instruction = "Convert the following text from English to Hindi: \n\n{text}"

In [31]:
CUSTOM_SYSTEM_PROMPT="You are an advanced assistant that excels at translation."

In [32]:
SYSTEM_PROMPT=B_SYS+CUSTOM_SYSTEM_PROMPT+E_SYS

In [33]:
SYSTEM_PROMPT

'<<SYS>>\nYou are an advanced assistant that excels at translation.\n<<SYS>>\n\n'

In [34]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [35]:
print(template)

[INST]<<SYS>>
You are an advanced assistant that excels at translation.
<<SYS>>

Convert the following text from English to Hindi: 

{text}[/INST]


In [36]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [37]:
text ="How are you"

In [38]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [39]:
output = LLM_Chain.run(text)

In [40]:
output


'  Sure! Here\'s the translation of "How are you" from English to Hindi:\n\nकैसे हैं आप (Kaise Hain Aap)'

In [41]:
import textwrap
wrapped_text = textwrap.fill(output, width=100)
print(wrapped_text +'\n\n')


  Sure! Here's the translation of "How are you" from English to Hindi:  कैसे हैं आप (Kaise Hain Aap)




In [42]:
instruction = "Summarize the following text for me : {text}"

In [43]:
SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [44]:
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [45]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [46]:
print(template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Summarize the following text for me : {text}[/INST]


In [47]:
text = """
Consistency is what separates those who succeed from those who don't..

Here's a Four-pillar approach to help you stay consistent and achieve your goals:


1) Set clear goals: Setting clear goals is a crucial step in achieving success and personal growth.

2) Build constructive habits: Building habits takes time and consistency. Start Small i.e. begin with a simple, manageable habit that you can easily incorporate into your daily routine.

3) Staying accountable: Accountability can help you stay focused and motivated on your goals, increasing the likelihood of achieving them. Whether it's through setting milestones, tracking progress, or sharing your goals with someone, accountability can be a powerful tool for success.

4) Clarity in one's purpose: Having clarity in one's purpose can provide direction, motivation, and a sense of fulfillment in life. It helps you make informed decisions and stay focused on what truly matters to you.
"""

In [48]:
prompt=PromptTemplate(input_variables=["text"],
                      template=template)

In [49]:
LLM_Chain = LLMChain(prompt=prompt, llm=llm)

In [50]:
print(template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Summarize the following text for me : {text}[/INST]


In [51]:
LLM_Chain.run(text)

"  The text emphasizes the importance of consistency in achieving success and personal growth. It proposes a four-pillar approach to help individuals stay consistent and reach their goals:\n\n1. Set clear goals: Establishing specific, measurable, and achievable goals is crucial for success.\n2. Build constructive habits: Developing small, manageable habits can be incorporated into daily routines, leading to long-term progress.\n3. Staying accountable: Accountability through milestones, progress tracking, or sharing goals with someone can maintain motivation and focus.\n4. Clarity in one's purpose: Having a clear purpose provides direction, motivation, and fulfillment in life, enabling informed decisions and staying focused on what matters most.\n\nIn summary, the text highlights the significance of consistency and offers a practical approach to achieving goals and personal growth through setting clear objectives, building constructive habits, staying accountable, and maintaining a clea

In [52]:
instruction = "Summarize the following text for me : \n\n{text}"
Custom_SYSTEM_PROMPT = "You are an expert in summarizing and expressing key ideas succintly"
SYSTEM_PROMPT = B_SYS + Custom_SYSTEM_PROMPT + E_SYS
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [53]:
print(template)

[INST]<<SYS>>
You are an expert in summarizing and expressing key ideas succintly
<<SYS>>

Summarize the following text for me : 

{text}[/INST]


In [54]:
prompt=PromptTemplate(input_variables=["text"],
                      template=template)

In [55]:
LLM_Chain = LLMChain(prompt=prompt, llm=llm)

In [56]:
output = LLM_Chain.run(text)

In [57]:
output

"  Sure, here's a summary of the text in a few sentences:\nConsistency is key to achieving success and personal growth. A four-pillar approach can help you stay consistent and reach your goals, which include setting clear goals, building constructive habits, staying accountable, and having clarity in your purpose. By incorporating these pillars into your daily routine, you can increase your chances of success and achieve your goals."

In [58]:
import textwrap
wrapped_text = textwrap.fill(output, width=100)
print(wrapped_text +'\n\n')


  Sure, here's a summary of the text in a few sentences: Consistency is key to achieving success and
personal growth. A four-pillar approach can help you stay consistent and reach your goals, which
include setting clear goals, building constructive habits, staying accountable, and having clarity
in your purpose. By incorporating these pillars into your daily routine, you can increase your
chances of success and achieve your goals.




In [59]:
instruction = "Please provide the summary of the following book {book_name}\n"

In [60]:
B_INST, E_INST= "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [61]:
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""

In [62]:
SYSTEM_PROMPT=B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [63]:
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [64]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [65]:
print(template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Please provide the summary of the following book {book_name}
[/INST]


In [66]:
prompt = PromptTemplate(input_variables={'book_name'}, template=template)

In [67]:
LLM_Chain = LLMChain(llm=llm, prompt=prompt)

In [68]:
LLM_Chain.run("Atomic Habits")

'  Of course! I\'d be happy to help you with a summary of the book "Atomic Habits" by James Clear.\n"Atomic Habits" is a self-help book that provides practical strategies for building good habits and breaking bad ones. The author, James Clear, is a recognized expert in the field of habit formation and has a compelling story to share.\nHere\'s a brief summary of the book:\n1. The Power of Atomic Habits: The book begins by explaining the concept of "atomic habits," which are small habits that, when practiced consistently, can lead to significant improvements in our lives. Clear argues that these small habits are the building blocks of our larger habits and that by focusing on them, we can create lasting change.\n2. The Four Laws of Behavior Change: Clear identifies four laws of behavior change that can help us build good habits and break bad ones. These laws are: make it obvious, make it attractive, make it easy, and make it satisfying. By applying these laws, we can create habits that a

In [69]:
instruction = "Please provide the summary as bullet points for this book {book_name}. Each bullet point should appear in a new line"

In [70]:
#instruction = "Please provide the summary of the following book {book_name}"

In [71]:
B_INST, E_INST= "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [72]:
CUSTOM_SYSTEM_PROMPT="""\
You are an expert in Summarizing the text and extracting key information from it
"""

In [73]:
SYSTEM_PROMPT=B_SYS + CUSTOM_SYSTEM_PROMPT + E_SYS

In [74]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [75]:
print(template)

[INST]<<SYS>>
You are an expert in Summarizing the text and extracting key information from it

<<SYS>>

Please provide the summary as bullet points for this book {book_name}. Each bullet point should appear in a new line[/INST]


In [76]:
prompt = PromptTemplate(input_variables={'book_name'}, template=template)

In [77]:
LLM_Chain = LLMChain(llm=llm, prompt=prompt)

In [78]:
LLM_Chain.run("Rework")

'  Sure, here is a summary of the book "Rework" by Jason Fried and David Heinemeier Hansson in bullet points:\n\n• The traditional 9-to-5 workday is outdated and can lead to burnout, stress, and a lack of productivity.\n• The authors argue that the modern workplace should be flexible and adaptable to the needs of employees, rather than adhering to a rigid schedule.\n• They suggest that work should be focused on achieving specific goals, rather than simply filling time.\n• The authors believe that creativity and innovation are stifled by traditional work structures and processes, and that a more fluid approach to work can lead to better outcomes.\n• They argue that hierarchy and bureaucracy can hinder progress and productivity, and that a more flat organizational structure can be more effective.\n• The book provides examples of companies that have successfully adopted a more flexible and adaptable approach to work, such as Basecamp and the 37signals company.\n• The authors emphasize the

In [79]:
output = LLM_Chain.run("Rework")

In [81]:
import textwrap
wrapped_text = textwrap.fill(output, width=100)
output = wrapped_text +'\n\n'
output.strip().split("*")

['Sure! Here is a summary of "Rework" by Jason Fried and David Heinemeier Hansson in bullet points:\n• The traditional office environment is dead and has been replaced by remote work, flexible\nschedules, and a focus on output rather than input. • The traditional 9-to-5 workday is no longer\nrelevant, and employees should be allowed to work when they are most productive. • The concept of\n"work" is changing, and it\'s no longer about putting in a certain number of hours but rather about\nachieving specific goals. • Managers should focus on results rather than hours worked, and employees\nshould be given the freedom to work in a way that allows them to be most productive. • The\ntraditional hierarchical structure of companies is no longer effective, and companies should adopt a\nflatter, more decentralized structure. • Companies should focus on creating a culture of autonomy,\nrather than a culture of hierarchy. • The traditional approach to innovation, which involves\nbrainstorming ses

In [82]:
from langchain.memory import ConversationBufferMemory

In [83]:
instruction = "Chat History:\n\n{chat_history} \n User:\n {user_input}"

Custom_SYSTEM_PROMPT = """You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context"""

In [84]:
SYSTEM_PROMPT=B_SYS + Custom_SYSTEM_PROMPT + E_SYS

In [85]:
SYSTEM_PROMPT

'<<SYS>>\nYou are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context\n<<SYS>>\n\n'

In [86]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [88]:
print(template)

[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

{chat_history} 
 User:
 {user_input}[/INST]


In [89]:
prompt=PromptTemplate(input_variables=["chat_history", "user_input"], template=template)

In [90]:
memory=ConversationBufferMemory(memory_key="chat_history")

In [91]:
chain=LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

In [93]:
chain.run(user_input="Hello")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hrllo
AI:   Hello! How may I assist you today? 
 User:
 Hello[/INST]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


"  Of course! I'm here to help. How may I assist you today?"

In [94]:
chain.run(user_input="How is the weather today in South India")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hrllo
AI:   Hello! How may I assist you today?
Human: Hello
AI:   Of course! I'm here to help. How may I assist you today? 
 User:
 How is the weather today in South India[/INST]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


"  AI: The weather in South India is currently pleasant, with clear skies and temperatures ranging from 25-30 degrees Celsius (77-86 degrees Fahrenheit) in most areas. However, I must inform you that I'm just an AI and do not have access to real-time weather information or current conditions in specific locations. If you would like to know the latest weather updates, I suggest checking with a reliable weather source or visiting a local news website. Is there anything else I can assist you with?"

In [95]:
chain.run(user_input="If today is Saturday What number day of the week is that")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hrllo
AI:   Hello! How may I assist you today?
Human: Hello
AI:   Of course! I'm here to help. How may I assist you today?
Human: How is the weather today in South India
AI:   AI: The weather in South India is currently pleasant, with clear skies and temperatures ranging from 25-30 degrees Celsius (77-86 degrees Fahrenheit) in most areas. However, I must inform you that I'm just an AI and do not have access to real-time weather information or current conditions in specific locations. If you would like to know the latest weather updates, I suggest checking with a reliable weather source or visiting a local news website. Is there anything else I can assist you with? 
 User:
 If today is Saturday What number day of the week is that[/INST]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


"  AI: Good afternoon! I'm happy to help. Today is Saturday, which means it is the 5th day of the week."

In [96]:
chain.run(user_input="Which day of the week is tomorrow")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hrllo
AI:   Hello! How may I assist you today?
Human: Hello
AI:   Of course! I'm here to help. How may I assist you today?
Human: How is the weather today in South India
AI:   AI: The weather in South India is currently pleasant, with clear skies and temperatures ranging from 25-30 degrees Celsius (77-86 degrees Fahrenheit) in most areas. However, I must inform you that I'm just an AI and do not have access to real-time weather information or current conditions in specific locations. If you would like to know the latest weather updates, I suggest checking with a reliable weather source or visiting a local news website. Is there anything else I can assist you with?
Human: If today is Saturday What number day of the week is that
AI:   AI: Good afternoon! I'm happy 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


'  AI: Tomorrow is Sunday, which means it is the 6th day of the week.'

In [97]:
chain.run(user_input="My name is Anantha Narayanan please remember")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hrllo
AI:   Hello! How may I assist you today?
Human: Hello
AI:   Of course! I'm here to help. How may I assist you today?
Human: How is the weather today in South India
AI:   AI: The weather in South India is currently pleasant, with clear skies and temperatures ranging from 25-30 degrees Celsius (77-86 degrees Fahrenheit) in most areas. However, I must inform you that I'm just an AI and do not have access to real-time weather information or current conditions in specific locations. If you would like to know the latest weather updates, I suggest checking with a reliable weather source or visiting a local news website. Is there anything else I can assist you with?
Human: If today is Saturday What number day of the week is that
AI:   AI: Good afternoon! I'm happy 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


'  Of course! I will remember your name, Anantha Narayanan. How may I assist you today?'

In [98]:
chain.run(user_input="Can you tell me the meaning of my name Anantha Narayanan")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<<SYS>>

Chat History:

Human: Hrllo
AI:   Hello! How may I assist you today?
Human: Hello
AI:   Of course! I'm here to help. How may I assist you today?
Human: How is the weather today in South India
AI:   AI: The weather in South India is currently pleasant, with clear skies and temperatures ranging from 25-30 degrees Celsius (77-86 degrees Fahrenheit) in most areas. However, I must inform you that I'm just an AI and do not have access to real-time weather information or current conditions in specific locations. If you would like to know the latest weather updates, I suggest checking with a reliable weather source or visiting a local news website. Is there anything else I can assist you with?
Human: If today is Saturday What number day of the week is that
AI:   AI: Good afternoon! I'm happy 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


'  Certainly, Anantha Narayanan is a name with multiple meanings and origins. Here are some possible interpretations:\n1. Anantha is a Sanskrit word that means "endless" or "infinite". It is often associated with the Hindu deity Lord Vishnu, who is believed to have the power to take on infinite forms.\n2. Narayanan is a Tamil name that means "blessing" or "divine grace". It is often found in South Indian communities, particularly in Tamil Nadu.\nCombining the two names, Anantha Narayanan can be interpreted as "endless blessing" or "infinite divine grace". However, the exact meaning and origin of the name may vary depending on the cultural and linguistic context. Is there anything else I can help you with?'